In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pmdarima as pm
import statsmodels
from pmdarima import model_selection
import math
import pickle
from sklearn.metrics import mean_squared_error, mean_absolute_error
from statsmodels.tsa.statespace.sarimax import SARIMAX
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler, StandardScaler

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

Fit SARIMA model

In [ ]:
freq = 60
df = pd.read_csv("../data/worldcup98_may_minute.csv", index_col=0, parse_dates=True)
downsampled_df = df.resample(str(freq) + "T").mean()
downsampled_df

In [ ]:
freq = 60
seasonal_order = (24 * 60) // freq
split = 0.8

raw_data = np.asarray(downsampled_df["count"])
train_size = math.floor(len(raw_data) * split)
train, test = model_selection.train_test_split(raw_data, train_size=train_size)
plt.plot(downsampled_df)
plt.show()

In [ ]:
p,d,q = (4,1,2)
P,D,Q = (4,1,1)
m = (24 * 60) / 60

def fit_sarima(train_data, freq=60, params=(p,d,q), seasonal_params=(P,D,Q)):   #ARIMA(4,1,0)(2,1,0)[24] 
    seasonal_period = (24 * 60) // freq
    seasonal_order = seasonal_params + (seasonal_period,)
    print(f"Fitting SARIMA")
    start = datetime.now()
    model = SARIMAX(train_data, order=params, seasonal_order=seasonal_order)
    model_fit = model.fit()
    end = datetime.now()
    print(f"Fit SARIMA in {str(end-start)}")
    return model_fit

In [ ]:
def build_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, activation='relu', input_shape=input_shape))
    model.add(Dropout(0.2))
    model.add(LSTM(96))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

In [ ]:
def fit_lstm(residuals, n_lags=5, epochs=10):
    X, y = [], []
    for i in range(n_lags, len(residuals)):
        X.append(residuals[i - n_lags:i])
        y.append(residuals[i])
    X, y = np.array(X), np.array(y)
    X = X.reshape((X.shape[0], X.shape[1], 1))
    
    model = build_lstm_model((X.shape[1], 1))
    print(f"Fitting LSTM")
    start = datetime.now()
    model.fit(X, y, epochs=epochs, verbose=0)
    end = datetime.now()
    print(f"Fit LSTM in {str(end-start)}")
    return model
  

In [ ]:
len(downsampled_df)

In [ ]:
from sklearn.metrics import mean_squared_error

window_size = 696
n_lags = 10
epochs = 20
refit = False
data = downsampled_df["count"].values

actuals = []
hybrid_predictions = []
sarima_predictions = []
naive_predictions = []

lstm_residual_predictions = []
residual_actuals = []



for t in range(window_size, len(data)):
  print(f"{t+1} / {len(data)}")
  
  train_data = data[t - window_size : t]
  actual = data[t]
  
  naive_predictions.append(data[t-1])
  
  
  # Fit SARIMA
  if t == window_size:
    sarima_model = fit_sarima(train_data)
  sarima_forecast = sarima_model.forecast(steps=1)[0]
  sarima_predictions.append(sarima_forecast)
  
  # Get residuals before adding actual observation
  residuals = sarima_model.resid.reshape(-1,1)
  residual_actuals.append(actual - sarima_forecast) # är detta verkligen korrekt?
  
  sarima_model = sarima_model.append([actual], refit=refit)
  
  
  # Scale residuals before passing to LSTM
  scaler = StandardScaler()
  residuals_scaled = scaler.fit_transform(residuals)
  
  
  # Fit LSTM on residuals
  
  lstm_fit = fit_lstm(residuals_scaled, n_lags=n_lags, epochs=epochs)
  X_input = residuals_scaled[-n_lags:].reshape(1, n_lags, 1)
  
  # lstm_fit = fit_lstm(scaled_residuals, n_lags=n_lags, epochs=epochs)
  
  X_input = residuals_scaled[-n_lags:].reshape(1, n_lags, 1)
  
  # X_input = scaler.transform(residuals[-n_lags:])
  
  lstm_residual_prediction = lstm_fit.predict(X_input)[0][0]
  
  # Inverse transform of LSTM output
  lstm_residual_prediction = scaler.inverse_transform([[lstm_residual_prediction]])[0][0]
  lstm_residual_predictions.append(lstm_residual_prediction)
  
  
  
  final_forecast = sarima_forecast + lstm_residual_prediction
  
  hybrid_predictions.append(final_forecast)
  actuals.append(actual)

In [ ]:
sarima_model.summary()

In [ ]:
naive_predictions

In [ ]:
actuals

In [ ]:
sarima_predictions

In [ ]:
hybrid_mse = mean_squared_error(hybrid_predictions, actuals)
naive_mse = mean_squared_error(naive_predictions, actuals)
sarima_mse = mean_squared_error(sarima_predictions, actuals)

hybrid_mae = mean_absolute_error(hybrid_predictions, actuals)
naive_mae = mean_absolute_error(naive_predictions, actuals)
sarima_mae = mean_absolute_error(sarima_predictions, actuals)

print("hybrid_mae", hybrid_mae)
print("naive_mae", naive_mae)
print("sarima_mae", sarima_mae)


plt.plot(actuals, label="Actual")
plt.plot(hybrid_predictions, label="Hybrid")
plt.plot(naive_predictions, label="Naive")
plt.plot(sarima_predictions, label="SARIMA")
plt.xlabel("Time  (Hour)")
plt.ylabel("Requests")
plt.legend()

mse_text = (
    f"Hybrid MSE: {hybrid_mse:.2f}\n"
    f"Naive MSE: {naive_mse:.2f}\n"
    f"SARIMA MSE: {sarima_mse:.2f}"
)
plt.text(0.01, 0.95, mse_text, transform=plt.gca().transAxes,
         fontsize=10, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.7))

plt.tight_layout()
filename= f"figures/ARIMA({p},{d},{q})({P},{D},{Q})-norefit-onestep-scaled.png"
plt.savefig(filename, dpi=300)
print(f"Saved figure to {filename}")
plt.show()

In [ ]:
plt.plot(sarima_model.resid)


In [ ]:
plt.plot(residual_actuals, label="Actual")
plt.plot(lstm_residual_predictions, label="LSTM predict")
plt.xlabel("Time (Hours)")
plt.ylabel("Residual")
plt.legend()
mse_text = (
    f"MSE: {hybrid_mse:.2f}\n"
)
plt.text(0.01, 0.95, mse_text, transform=plt.gca().transAxes,
         fontsize=10, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.7))
filename= f"figures/Residuals({p},{d},{q})({P},{D},{Q})-norefit-onestep-scaled.png"
plt.savefig(filename, dpi=300)
print(f"Saved figure to {filename}")
print(mean_squared_error(lstm_residual_predictions, residual_actuals))

In [ ]:
b = np.array([2,3,4])

a = StandardScaler()
a.fit_transform(b.reshape(-1,1))[0][0]